In [ ]:
import numpy as np
import cv2
from google.colab import drive
import os
from PIL import Image

drive.mount('/content/drive')

# input and output directories
input_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset'
output_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset CLAHE'

# output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def apply_clahe(image):
    # Create CLAHE object
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

    # Apply CLAHE
    enhanced = clahe.apply(image.astype(np.uint8))

    # Normalize result to 0-255 range
    enhanced = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
    return enhanced

# Process images
for filename in os.listdir(input_dir):
    if filename.endswith('.png'):
        # Read image
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply CLAHE
        processed_img = apply_clahe(img)

        # Save processed image
        output_path = os.path.join(output_dir, f'clahe_{filename}')
        cv2.imwrite(output_path, processed_img)

print("CLAHE enhancement completed")
print(f'Total images processed: {len([name for name in os.listdir(output_dir) if name.endswith(".png")])}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CLAHE enhancement completed
Total images processed: 480


In [ ]:
!pip install fpdf2

import numpy as np
import cv2
import os
from PIL import Image
from fpdf import FPDF
import matplotlib.pyplot as plt
from datetime import datetime

def calculate_snr(img):
    """Calculate Signal-to-Noise Ratio"""
    img_float = img.astype(float)

    # Define ROI for signal defects are brighter
    threshold = np.mean(img_float) + np.std(img_float)
    signal_mask = img_float > threshold
    noise_mask = ~signal_mask

    if np.sum(signal_mask) > 0 and np.sum(noise_mask) > 0:
        signal_mean = np.mean(img_float[signal_mask])
        noise_std = np.std(img_float[noise_mask])
        snr = 20 * np.log10(signal_mean / noise_std) if noise_std != 0 else 0
    else:
        snr = 0

    return snr

def create_comparison_plot(original_img, processed_img, snr_orig, snr_proc, filename):
    """Create a comparison plot with SNR values"""
    plt.figure(figsize=(12, 5))

    # Plot original image
    plt.subplot(121)
    plt.imshow(original_img, cmap='gray')
    plt.title(f'Original (SNR: {snr_orig:.2f} dB)')
    plt.axis('off')

    # Plot processed image
    plt.subplot(122)
    plt.imshow(processed_img, cmap='gray')
    plt.title(f'CLAHE Enhanced (SNR: {snr_proc:.2f} dB)')
    plt.axis('off')

    plt.savefig(f'temp_{filename}.png', bbox_inches='tight', dpi=300)
    plt.close()

# Define directories
original_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset'
processed_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset CLAHE'
output_dir = '/content/drive/My Drive/Colab Notebooks/analysis_results'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Title page
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'CLAHE Enhancement SNR Analysis Report', ln=True, align='C')
pdf.set_font('Arial', '', 12)
pdf.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', ln=True, align='C')

# Initialize lists to store metrics
all_metrics = []

# Process images
print("Processing images...")
for filename in sorted(os.listdir(original_dir)):
    if filename.endswith('.png'):
        # Read images
        original_path = os.path.join(original_dir, filename)
        processed_path = os.path.join(processed_dir, f'clahe_{filename}')

        if os.path.exists(processed_path):
            original_img = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
            processed_img = cv2.imread(processed_path, cv2.IMREAD_GRAYSCALE)

            # Calculate SNR
            snr_original = calculate_snr(original_img)
            snr_processed = calculate_snr(processed_img)

            # Store metrics
            all_metrics.append({
                'filename': filename,
                'SNR_original': snr_original,
                'SNR_processed': snr_processed,
                'SNR_improvement': snr_processed - snr_original
            })

            # Create comparison plot
            create_comparison_plot(original_img, processed_img, snr_original, snr_processed, filename)

            # Add to PDF
            pdf.add_page()
            pdf.set_font('Arial', 'B', 12)
            pdf.cell(0, 10, f'Image: {filename}', ln=True)
            pdf.image(f'temp_{filename}.png', x=10, w=190)

            # Add SNR values
            pdf.set_font('Arial', '', 10)
            pdf.cell(0, 10, f'SNR Improvement: {snr_processed - snr_original:.2f} dB', ln=True)

            # Clean up temporary files
            os.remove(f'temp_{filename}.png')

            print(f"Processed {filename}")

# Add summary statistics
pdf.add_page()
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, 'Summary Statistics', ln=True)
pdf.set_font('Arial', '', 12)

# Calculate summary statistics
avg_snr_improvement = np.mean([m['SNR_improvement'] for m in all_metrics])
max_snr_improvement = np.max([m['SNR_improvement'] for m in all_metrics])
min_snr_improvement = np.min([m['SNR_improvement'] for m in all_metrics])

# Add statistics to PDF
pdf.cell(0, 10, f'Average SNR Improvement: {avg_snr_improvement:.2f} dB', ln=True)
pdf.cell(0, 10, f'Maximum SNR Improvement: {max_snr_improvement:.2f} dB', ln=True)
pdf.cell(0, 10, f'Minimum SNR Improvement: {min_snr_improvement:.2f} dB', ln=True)

# Create SNR improvement distribution plot
plt.figure(figsize=(10, 6))
improvements = [m['SNR_improvement'] for m in all_metrics]
plt.hist(improvements, bins=30, edgecolor='black')
plt.title('Distribution of SNR Improvements')
plt.xlabel('SNR Improvement (dB)')
plt.ylabel('Number of Images')
plt.savefig('snr_distribution.png', bbox_inches='tight', dpi=300)
plt.close()

# Add distribution plot to PDF
pdf.add_page()
pdf.image('snr_distribution.png', x=10, w=190)
os.remove('snr_distribution.png')

# Save PDF
pdf_path = os.path.join(output_dir, 'CLAHE_snr_analysis.pdf')
pdf.output(pdf_path)

print(f"\nAnalysis completed. PDF report saved to: {pdf_path}")

# Print overall statistics
print(f"\nSummary Statistics:")
print(f"Average SNR Improvement: {avg_snr_improvement:.2f} dB")
print(f"Maximum SNR Improvement: {max_snr_improvement:.2f} dB")
print(f"Minimum SNR Improvement: {min_snr_improvement:.2f} dB")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 kB 10.3 MB/s eta 0:00:00


<ipython-input-2-52afce842005>:62: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
<ipython-input-2-52afce842005>:63: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, 'CLAHE Enhancement SNR Analysis Report', ln=True, align='C')
<ipython-input-2-52afce842005>:64: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
<ipython-input-2-52afce842005>:65: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', ln=True, align='C')


Processing images...


<ipython-input-2-52afce842005>:99: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-2-52afce842005>:100: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Image: {filename}', ln=True)
<ipython-input-2-52afce842005>:104: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)
<ipython-input-2-52afce842005>:105: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'SNR Improvement: {snr_processed - snr_original:.2f} dB', ln=True)


Processed P1X1_train_norm_image_0000.png
Processed P1X1_train_norm_image_0001.png
Processed P1X1_train_norm_image_0002.png
Processed P1X1_train_norm_image_0003.png
Processed P1X1_train_norm_image_0004.png
Processed P1X1_train_norm_image_0005.png
Processed P1X1_train_norm_image_0006.png
Processed P1X1_train_norm_image_0007.png
Processed P1X1_train_norm_image_0008.png
Processed P1X1_train_norm_image_0009.png
Processed P1X1_train_norm_image_0010.png
Processed P1X1_train_norm_image_0011.png
Processed P1X1_train_norm_image_0012.png
Processed P1X1_train_norm_image_0013.png
Processed P1X1_train_norm_image_0014.png
Processed P1X1_train_norm_image_0015.png
Processed P1X1_train_norm_image_0016.png
Processed P1X1_train_norm_image_0017.png
Processed P1X1_train_norm_image_0018.png
Processed P1X1_train_norm_image_0019.png
Processed P1X1_train_norm_image_0020.png
Processed P1X1_train_norm_image_0021.png
Processed P1X1_train_norm_image_0022.png
Processed P1X1_train_norm_image_0023.png
Processed P1X1_t

<ipython-input-2-52afce842005>:114: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 14)
<ipython-input-2-52afce842005>:115: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, 'Summary Statistics', ln=True)
<ipython-input-2-52afce842005>:116: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
<ipython-input-2-52afce842005>:124: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Average SNR Improvement: {avg_snr_improvement:.2f} dB', ln=True)
<ipython-input-2-52afce842005>:125: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new


Analysis completed. PDF report saved to: /content/drive/My Drive/Colab Notebooks/analysis_results/CLAHE_snr_analysis.pdf

Summary Statistics:
Average SNR Improvement: -0.81 dB
Maximum SNR Improvement: 0.95 dB
Minimum SNR Improvement: -7.36 dB


In [ ]:
import numpy as np
import cv2
from google.colab import drive
import os
from PIL import Image

# Input and output directories
input_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset'
output_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset Bilateral'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def apply_bilateral(image):
    # Apply bilateral filter
    # Parameters: image, diameter of pixel neighborhood, sigma color, sigma space
    enhanced = cv2.bilateralFilter(image, d=9, sigmaColor=75, sigmaSpace=75)

    # Normalize result to 0-255 range
    enhanced = cv2.normalize(enhanced, None, 0, 255, cv2.NORM_MINMAX)
    return enhanced

# Process all images
for filename in os.listdir(input_dir):
    if filename.endswith('.png'):
        # Read image
        img_path = os.path.join(input_dir, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        # Apply Bilateral Filter
        processed_img = apply_bilateral(img)

        # Save processed image
        output_path = os.path.join(output_dir, f'bilateral_{filename}')
        cv2.imwrite(output_path, processed_img)

print("Bilateral filtering completed")
print(f'Total images processed: {len([name for name in os.listdir(output_dir) if name.endswith(".png")])}')

Bilateral filtering completed
Total images processed: 480


In [ ]:
import numpy as np
import cv2
import os
from PIL import Image
from fpdf import FPDF
import matplotlib.pyplot as plt
from datetime import datetime

def calculate_snr(img):
    """Calculate Signal-to-Noise Ratio"""
    img_float = img.astype(float)

    # Define ROI for signal, defects are brighter
    threshold = np.mean(img_float) + np.std(img_float)
    signal_mask = img_float > threshold
    noise_mask = ~signal_mask

    if np.sum(signal_mask) > 0 and np.sum(noise_mask) > 0:
        signal_mean = np.mean(img_float[signal_mask])
        noise_std = np.std(img_float[noise_mask])
        snr = 20 * np.log10(signal_mean / noise_std) if noise_std != 0 else 0
    else:
        snr = 0

    return snr

def create_comparison_plot(original_img, processed_img, snr_orig, snr_proc, filename):
    """Create a comparison plot with SNR values"""
    plt.figure(figsize=(12, 5))

    # Plot original image
    plt.subplot(121)
    plt.imshow(original_img, cmap='gray')
    plt.title(f'Original (SNR: {snr_orig:.2f} dB)')
    plt.axis('off')

    # Plot processed image
    plt.subplot(122)
    plt.imshow(processed_img, cmap='gray')
    plt.title(f'Bilateral Filtered (SNR: {snr_proc:.2f} dB)')
    plt.axis('off')

    plt.savefig(f'temp_{filename}.png', bbox_inches='tight', dpi=300)
    plt.close()

# Define directories
original_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset'
processed_dir = '/content/drive/My Drive/Colab Notebooks/thermal img dataset Bilateral'
output_dir = '/content/drive/My Drive/Colab Notebooks/analysis_results'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Create PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)

# Add title page
pdf.add_page()
pdf.set_font('Arial', 'B', 16)
pdf.cell(0, 10, 'Bilateral Filtering SNR Analysis Report', ln=True, align='C')
pdf.set_font('Arial', '', 12)
pdf.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', ln=True, align='C')

# Initialize lists to store metrics
all_metrics = []

# Process images
print("Processing images...")
for filename in sorted(os.listdir(original_dir)):
    if filename.endswith('.png'):
        # Read images
        original_path = os.path.join(original_dir, filename)
        processed_path = os.path.join(processed_dir, f'bilateral_{filename}')

        if os.path.exists(processed_path):
            original_img = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
            processed_img = cv2.imread(processed_path, cv2.IMREAD_GRAYSCALE)

            # Calculate SNR
            snr_original = calculate_snr(original_img)
            snr_processed = calculate_snr(processed_img)

            # Store metrics
            all_metrics.append({
                'filename': filename,
                'SNR_original': snr_original,
                'SNR_processed': snr_processed,
                'SNR_improvement': snr_processed - snr_original
            })

            # Create comparison plot
            create_comparison_plot(original_img, processed_img, snr_original, snr_processed, filename)

            # Add to PDF
            pdf.add_page()
            pdf.set_font('Arial', 'B', 12)
            pdf.cell(0, 10, f'Image: {filename}', ln=True)
            pdf.image(f'temp_{filename}.png', x=10, w=190)

            # Add SNR values
            pdf.set_font('Arial', '', 10)
            pdf.cell(0, 10, f'SNR Improvement: {snr_processed - snr_original:.2f} dB', ln=True)

            # Clean up temporary files
            os.remove(f'temp_{filename}.png')

            print(f"Processed {filename}")

# Add summary statistics
pdf.add_page()
pdf.set_font('Arial', 'B', 14)
pdf.cell(0, 10, 'Summary Statistics', ln=True)
pdf.set_font('Arial', '', 12)

# Calculate summary statistics
avg_snr_improvement = np.mean([m['SNR_improvement'] for m in all_metrics])
max_snr_improvement = np.max([m['SNR_improvement'] for m in all_metrics])
min_snr_improvement = np.min([m['SNR_improvement'] for m in all_metrics])

# Add statistics to PDF
pdf.cell(0, 10, f'Average SNR Improvement: {avg_snr_improvement:.2f} dB', ln=True)
pdf.cell(0, 10, f'Maximum SNR Improvement: {max_snr_improvement:.2f} dB', ln=True)
pdf.cell(0, 10, f'Minimum SNR Improvement: {min_snr_improvement:.2f} dB', ln=True)

# Create SNR improvement distribution plot
plt.figure(figsize=(10, 6))
improvements = [m['SNR_improvement'] for m in all_metrics]
plt.hist(improvements, bins=30, edgecolor='black')
plt.title('Distribution of SNR Improvements')
plt.xlabel('SNR Improvement (dB)')
plt.ylabel('Number of Images')
plt.savefig('snr_distribution.png', bbox_inches='tight', dpi=300)
plt.close()

# Add distribution plot to PDF
pdf.add_page()
pdf.image('snr_distribution.png', x=10, w=190)
os.remove('snr_distribution.png')

# Save PDF
pdf_path = os.path.join(output_dir, 'Bilateral_snr_analysis.pdf')
pdf.output(pdf_path)

print(f"\nAnalysis completed. PDF report saved to: {pdf_path}")

# Print overall statistics
print(f"\nSummary Statistics:")
print(f"Average SNR Improvement: {avg_snr_improvement:.2f} dB")
print(f"Maximum SNR Improvement: {max_snr_improvement:.2f} dB")
print(f"Minimum SNR Improvement: {min_snr_improvement:.2f} dB")

<ipython-input-4-d22370668f05>:60: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 16)
<ipython-input-4-d22370668f05>:61: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, 'Bilateral Filtering SNR Analysis Report', ln=True, align='C')
<ipython-input-4-d22370668f05>:62: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
<ipython-input-4-d22370668f05>:63: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Generated on {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}', ln=True, align='C')


Processing images...


<ipython-input-4-d22370668f05>:97: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 12)
<ipython-input-4-d22370668f05>:98: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Image: {filename}', ln=True)
<ipython-input-4-d22370668f05>:102: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 10)
<ipython-input-4-d22370668f05>:103: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'SNR Improvement: {snr_processed - snr_original:.2f} dB', ln=True)


Processed P1X1_train_norm_image_0000.png
Processed P1X1_train_norm_image_0001.png
Processed P1X1_train_norm_image_0002.png
Processed P1X1_train_norm_image_0003.png
Processed P1X1_train_norm_image_0004.png
Processed P1X1_train_norm_image_0005.png
Processed P1X1_train_norm_image_0006.png
Processed P1X1_train_norm_image_0007.png
Processed P1X1_train_norm_image_0008.png
Processed P1X1_train_norm_image_0009.png
Processed P1X1_train_norm_image_0010.png
Processed P1X1_train_norm_image_0011.png
Processed P1X1_train_norm_image_0012.png
Processed P1X1_train_norm_image_0013.png
Processed P1X1_train_norm_image_0014.png
Processed P1X1_train_norm_image_0015.png
Processed P1X1_train_norm_image_0016.png
Processed P1X1_train_norm_image_0017.png
Processed P1X1_train_norm_image_0018.png
Processed P1X1_train_norm_image_0019.png
Processed P1X1_train_norm_image_0020.png
Processed P1X1_train_norm_image_0021.png
Processed P1X1_train_norm_image_0022.png
Processed P1X1_train_norm_image_0023.png
Processed P1X1_t

<ipython-input-4-d22370668f05>:112: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', 'B', 14)
<ipython-input-4-d22370668f05>:113: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, 'Summary Statistics', ln=True)
<ipython-input-4-d22370668f05>:114: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font('Arial', '', 12)
<ipython-input-4-d22370668f05>:122: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, f'Average SNR Improvement: {avg_snr_improvement:.2f} dB', ln=True)
<ipython-input-4-d22370668f05>:123: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new


Analysis completed. PDF report saved to: /content/drive/My Drive/Colab Notebooks/analysis_results/Bilateral_snr_analysis.pdf

Summary Statistics:
Average SNR Improvement: -1.40 dB
Maximum SNR Improvement: 0.88 dB
Minimum SNR Improvement: -20.17 dB
